# Create GTEx-TCGA DESeq Data Sets

```
Andrew Davidson
aedavids@ucsc.edu
```

TODO:
- start with test. ie small fail fast
- download col data to local disk
- download matrix to disk
- load col data
- load groupby


In [1]:
# use display() to print an html version of a data frame
# useful if dataFrame output is not generated by last like of cell
from IPython.display import display

import os
from os.path import exists
import pandas as pd


WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']
print(WORKSPACE_BUCKET)

gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275


In [2]:
tmp = "./tmp"
!mkdir -p $tmp

In [3]:
TCGAColDataURL = WORKSPACE_BUCKET + "/data/colData/trainingDataSets"
TCGADataURL    = WORKSPACE_BUCKET + "/data/matrices/groupByGeneId/trainingDataSets"

GTExColDataURL = WORKSPACE_BUCKET #+ "/GTEx*ColData.csv"
GTExDataURL    = WORKSPACE_BUCKET #+ "/GTEx*GroupByGenesCountMatrix.csv"

In [4]:
!gsutil ls $TCGAColDataURL

gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/colData/trainingDataSets/TCGA-TestColData.csv
gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/colData/trainingDataSets/TCGA-TrainColData.csv
gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/colData/trainingDataSets/TCGA-ValidateColData.csv
gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/colData/trainingDataSets/TCGA-miscColData.csv


In [5]:
!gsutil ls $TCGADataURL

gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/matrices/groupByGeneId/trainingDataSets/TCGA-TestGroupByGeneId.csv
gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/matrices/groupByGeneId/trainingDataSets/TCGA-TrainGroupByGeneId.csv
gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/matrices/groupByGeneId/trainingDataSets/TCGA-ValidateGroupByGeneId.csv


In [6]:
p = GTExDataURL + "/GTEx*GroupByGenesCountMatrix.csv"
!gsutil ls $p

gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/GTExTestGroupByGenesCountMatrix.csv
gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/GTExTrainGroupByGenesCountMatrix.csv
gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/GTExValidateGroupByGenesCountMatrix.csv


In [7]:
p = GTExColDataURL + "/GTEx*ColData.csv"
#print( p)
!gsutil ls $p

gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/GTExTestColData.csv
gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/GTExTrainColData.csv
gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/GTExValidateColData.csv


In [8]:
# def getColDataLocalPath(dataset="TCGA", trainingSetName="Test", cacheDir):
#     return = cacheDir +  "/TCGA-" + trainingSetName + "ColData.csv"

# def getGroupByLocalPath(dataset="TCGA", trainingSetName="Test", cacheDir):
#     return = cacheDir +  "/TCGA-" + trainingSetName + "ColData.csv"


In [9]:
# def downLoadColData(TCGAColDataURL, GTExColDataURL, cacheDir, trainingSetName="Test" ):
#     tcga = "/TCGA-" + trainingSetName + "ColData.csv"
#     if not exists(tcga):    
#         srcURL = TCGAColDataURL + tcga
#         print("srcURL:{} "

In [10]:
def downLoadColData(TCGAColDataURL, GTExColDataURL, cacheDir, trainingSetName="Test" ):
    tcga = "/TCGA-" + trainingSetName + "ColData.csv"
    if not exists(cacheDir + tcga):    
        srcURL = TCGAColDataURL + tcga
        print("srcURL:{} ".format(srcURL))
        !gsutil -m cp  $srcURL $cacheDir
    else:
        print("{} found in local cache".format(tcga))
    
    gtex = "/GTEx" + trainingSetName + "ColData.csv"
    if not exists(cacheDir + gtex):
        print("srcURL:{} ".format(srcURL))
        srcURL = GTExColDataURL + gtex
        !gsutil -m cp  $srcURL $cacheDir
    else:
        print("{} found in local cache".format(gtex))


downLoadColData(TCGAColDataURL, GTExColDataURL, tmp, trainingSetName="Test" )

/TCGA-TestColData.csv found in local cache
/GTExTestColData.csv found in local cache


In [11]:
def downLoadGroubByData(TCGADataURL, GTExDataURL, cacheDir, trainingSetName="Test"):
    tcga = "/TCGA-" + trainingSetName + "GroupByGeneId.csv"
    if not exists(cacheDir + tcga):    
        srcURL = TCGADataURL + tcga
        !gsutil -m cp  $srcURL $cacheDir
    else:
        print("{} found in local cache".format(tcga))

    
    gtex = "/GTEx" + trainingSetName + "GroupByGenesCountMatrix.csv"
    if not exists(cacheDir + gtex):
        srcURL = GTExDataURL + gtex
        !gsutil -m cp  $srcURL $cacheDir
    else:
        print("{} found in local cache".format(gtex))
        

downLoadGroubByData(TCGADataURL, GTExDataURL, tmp, trainingSetName="Test")

/TCGA-TestGroupByGeneId.csv found in local cache
/GTExTestGroupByGenesCountMatrix.csv found in local cache


In [12]:
def loadDF(localPath):
    retDF = None
    if not exists(localPath):
        print("ERROR {} not found".format(localPath))
    else:
        retDF = pd.read_csv(localPath)
    
    return retDF
        
def readData(cacheDir,  trainingSetName="Test"):
    GTexPath = cacheDir +  "/GTEx" + trainingSetName + "ColData.csv"
    GTExColDataDF = loadDF(GTexPath)
    
    TCGAPath = cacheDir +  "/TCGA-" + trainingSetName + "ColData.csv"
    TCGAColDataDF = loadDF(TCGAPath)
    
    GTEXGroupbyPath = cacheDir +  "/GTEx" + trainingSetName + "GroupByGenesCountMatrix.csv"
    GTExGroupByDF = loadDF(GTEXGroupbyPath)
    
    TCGAGroupbyPath = cacheDir + "/TCGA-" + trainingSetName + "GroupByGeneId.csv"
    TCGAGroupByDF = loadDF(TCGAGroupbyPath)
    
    return (GTExColDataDF, GTExGroupByDF, TCGAColDataDF, TCGAGroupByDF)

GTExColDataDF, GTExGroupByDF, TCGAColDataDF, TCGAGroupByDF = readData(tmp, trainingSetName="Test")

In [13]:
for df in [GTExColDataDF, GTExGroupByDF, TCGAColDataDF, TCGAGroupByDF]:
    print(type(df))
    print(df.shape)
    print()
    

print( display(GTExColDataDF.iloc[0:3, 0:3]) )
TCGAColDataDF.iloc[0:3, 0:3]
print( GTExGroupByDF.iloc[0:3, 0:3] )
TCGAGroupByDF.iloc[0:3, 0:3]

<class 'pandas.core.frame.DataFrame'>
(3471, 6)

<class 'pandas.core.frame.DataFrame'>
(74777, 3472)

<class 'pandas.core.frame.DataFrame'>
(1800, 7)

<class 'pandas.core.frame.DataFrame'>
(74777, 1801)



,sample_id,participant_id,tissue_id
0,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F,Adipose_Subcutaneous
1,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F,Artery_Tibial
2,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F,Artery_Coronary


None
       geneId  GTEX-1117F-0226-SM-5GZZ7  GTEX-1117F-0526-SM-5EGHJ
0        (A)n                         9                         1
1      (AAA)n                         0                         0
2  (AAAAAAC)n                         0                         0


,geneId,ACC-OR-A5J5-TP,ACC-OR-A5J9-TP
0,(A)n,11,14
1,(AAA)n,0,0
2,(AAAAAAC)n,0,0


In [51]:
def validateDataSets(colDataDF, groupByDF):
    print( "colDataDF.shape: {}".format(colDataDF.shape) )
    print( display(colDataDF.iloc[0:3, 0:2]) )
    
    print( "\ngroupByDF.shape: {}".format(groupByDF.shape) )
    print( display(groupByDF.iloc[0:3, 0:3]) )
    
    nRows = colDataDF.shape[0]
    nCols = groupByDF.shape[1]
    # DESeq requires cols and rows match. groupBy has an extra 'geneId' col data
    assert nRows == nCols -1, "ERROR num colData rows must equal num groupByCols -1"
    
    # make sure the rows and cols are in the same order
    colDataSampleNamesSeries = colDataDF.loc[:, 'sample_id']
    # skip the 'geneId col'
    groupBySampleNameList = groupByDF.columns.to_list()[1:]
    assert (colDataSampleNamesSeries == groupBySampleNameList).all(), "ERROR colDataSamples are not in same order as groupBy"

validateDataSets(GTExColDataDF, GTExGroupByDF)
validateDataSets(TCGAColDataDF, TCGAGroupByDF )

colDataDF.shape: (3471, 6)


,sample_id,participant_id
0,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F
1,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F
2,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F


None

groupByDF.shape: (74777, 3472)


,geneId,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0526-SM-5EGHJ
0,(A)n,9,1
1,(AAA)n,0,0
2,(AAAAAAC)n,0,0


None
colDataDF.shape: (1800, 7)


,sample_id,participant_id
0,ACC-OR-A5J5-TP,ACC-OR-A5J5
1,ACC-OR-A5J9-TP,ACC-OR-A5J9
2,ACC-OR-A5JA-TP,ACC-OR-A5JA


None

groupByDF.shape: (74777, 1801)


,geneId,ACC-OR-A5J5-TP,ACC-OR-A5J9-TP
0,(A)n,11,14
1,(AAA)n,0,0
2,(AAAAAAC)n,0,0


None


In [46]:
colDataSeries = GTExColDataDF.loc[:, 'sample_id'].head()
print( colDataSeries )
print()
# skip the 'geneId col'
groupBySampleName = GTExGroupByDF.columns.to_list()[1:6]
print( foo )
# print(GTExGroupByDF.index[0:5].to_list())

0    GTEX-1117F-0226-SM-5GZZ7
1    GTEX-1117F-0526-SM-5EGHJ
2    GTEX-1117F-0626-SM-5N9CS
3    GTEX-1117F-1326-SM-5EGHH
4    GTEX-1117F-2526-SM-5GZY6
Name: sample_id, dtype: object

['GTEX-1117F-0226-SM-5GZZ7', 'GTEX-1117F-0526-SM-5EGHJ', 'GTEX-1117F-0626-SM-5N9CS', 'GTEX-1117F-1326-SM-5EGHH', 'GTEX-1117F-2526-SM-5GZY6']


In [47]:
colDataSeries == foo

0    True
1    True
2    True
3    True
4    True
Name: sample_id, dtype: bool